<a href="https://colab.research.google.com/github/mgnarag/binarization-autoencoder/blob/main/Traditional_binarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

!ls drive/My\ Drive
file_path = "/content/drive/My Drive/Architectural_designs/one quadrant/"

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


from PIL import Image, ImageOps
Image.MAX_IMAGE_PIXELS = None
import os
import matplotlib.pyplot as plt

import cv2

In [12]:
def metrics(gt,thresh1):
  out_bin = np.where(thresh1 == 255, 1, thresh1)
  gt_bin = np.where(gt == 255, 1, gt)
  gt_bin_iou = gt_bin^1
  out_bin_iou = out_bin^1

  tp = np.sum(out_bin & gt_bin)
  fp = np.sum(out_bin & ~gt_bin)
  fn = np.sum(~out_bin & gt_bin)
  precision = tp / (tp + fp)
  recall = tp / (tp + fn)
  f1_score = (2*precision*recall)/(precision + recall)

  intersection = np.sum(np.logical_and(gt_bin_iou, out_bin_iou))
  union = np.sum(np.logical_or(gt_bin_iou, out_bin_iou))
  iou = intersection / union

  psnr = cv2.PSNR(gt, thresh1)

  return f1_score, iou, psnr


def simple_thresh(gt,test):
  F1 = []
  IOU = []
  PSNR = []
  for i in range(0,255):
    ret,thresh1 = cv2.threshold(test,i,255,cv2.THRESH_BINARY)
    M = metrics(gt,thresh1)
    F1.append(M[0])
    IOU.append(M[1])
    PSNR.append(M[2])
  index = np.nanargmax(F1)

  #saving:
  ret,thresh1 = cv2.threshold(test,index,255,cv2.THRESH_BINARY)
  final = Image.fromarray(thresh1)
  final.save(file_path + "/6 Traditional Thresh Results/"+"[1] SIMPLE_pix_" + str(index)+".png")

  return F1[index], IOU[index], PSNR[index], index

def otsu_thresh(gt,test):
  ret,thresh1 = cv2.threshold(test,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
  M = metrics(gt,thresh1)
  final = Image.fromarray(thresh1)
  final.save(file_path +  "/6 Traditional Thresh Results/"+"[2] OTSU.png")
  return M[0], M[1], M[2]

In [14]:
def adaptive_thresh_gaussian(gt,test):
  F1 = []
  IOU = []
  PSNR = []
  kernel = [i for i in range(3, 128, 4)]
  for k in kernel:
    F1_per_kernel = []
    IOU_per_kernel = []
    PSNR_per_kernel = []
    print(k)
    for i in range(0,100):
      #print(i)
      thresh1 = cv2.adaptiveThreshold(test,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,k,i)
      M = metrics(gt,thresh1)
      F1_per_kernel.append(M[0])
      IOU_per_kernel.append(M[1])
      PSNR_per_kernel.append(M[2])

    F1.append(F1_per_kernel)
    IOU.append(IOU_per_kernel)
    PSNR.append(PSNR_per_kernel)

  F1 = np.array(F1)
  flat_index = np.nanargmax(F1)
  index = np.unravel_index(flat_index, F1.shape)

  #saving:
  thresh1 = cv2.adaptiveThreshold(test,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,kernel[index[0]], index[1])
  final = Image.fromarray(thresh1)
  final.save(file_path + "/6 Traditional Thresh Results/"+ "[3] ADAPTIVE_GAUSSIAN_kernel_"+str(kernel[index[0]])+ "pix_"+str(index[1])+".png")

  return F1[index[0]][index[1]], IOU[index[0]][index[1]], PSNR[index[0]][index[1]], kernel[index[0]], index[1]

def adaptive_thresh_mean(gt,test):
  F1 = []
  IOU = []
  PSNR = []
  kernel = [i for i in range(3, 128, 4)]
  for k in kernel:
    F1_per_kernel = []
    IOU_per_kernel = []
    PSNR_per_kernel = []
    print(k)
    for i in range(0,100):
      thresh1 = cv2.adaptiveThreshold(test,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,k,i)
      #print(i)
      M = metrics(gt,thresh1)
      F1_per_kernel.append(M[0])
      IOU_per_kernel.append(M[1])
      PSNR_per_kernel.append(M[2])

    F1.append(F1_per_kernel)
    IOU.append(IOU_per_kernel)
    PSNR.append(PSNR_per_kernel)

  F1 = np.array(F1)
  flat_index = np.nanargmax(F1)
  index = np.unravel_index(flat_index, F1.shape)

  #saving:
  thresh1 = cv2.adaptiveThreshold(test,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,kernel[index[0]], index[1])
  final = Image.fromarray(thresh1)
  final.save(file_path +  "/6 Traditional Thresh Results/"+"[4] ADAPTIVE_MEAN_kernel_"+str(kernel[index[0]])+ "pix_"+str(index[1])+".png")

  return F1[index[0]][index[1]], IOU[index[0]][index[1]], PSNR[index[0]][index[1]], kernel[index[0]], index[1]

In [13]:
gt = cv2.imread(file_path+ '4 Testing/ground_truth.png', cv2.IMREAD_GRAYSCALE)
test = cv2.imread(file_path+ '4 Testing/1.png', cv2.IMREAD_GRAYSCALE)
#for simple thresh:
otsu = otsu_thresh(gt,test)
print('simple otsu = ', otsu)
simple = simple_thresh(gt,test)
print('simple thresh = ', simple)

simple otsu =  (0.570383542144578, 0.20839995626427807, 5.838131434810624)


<ipython-input-12-5df908de8908>:10: RuntimeWarning: invalid value encountered in scalar divide
  precision = tp / (tp + fp)


simple thresh =  (0.7091695462112346, 0.9376721800412845, 21.936912206820548, 48)


In [ ]:
#for adaptive thresh:
gaussian = adaptive_thresh_gaussian(gt,test)
print('adaptive thresh = ', gaussian)
mean = adaptive_thresh_mean(gt,test)
print('adaptive thresh = ', mean)

3


In [ ]:
simple_ = np.array(simple)
otsu_ = np.array(otsu)
gaussian_ = np.array(gaussian)
mean_ = np.array(mean)


# Creating DataFrames from arrays
df1 = pd.DataFrame({'Simple': simple_})
df2 = pd.DataFrame({'Otsu': otsu_})
df3 = pd.DataFrame({'Gaussian': gaussian_})
df4 = pd.DataFrame({'Mean': mean_})

# Concatenating DataFrames along columns (axis=1)
metric = pd.concat([df1, df2, df3, df4], axis=1)
metric = metric.T

headers = ['F1 score','IOU', 'PSNR', 'pixel/kerner', 'c parameter' ]
metric.columns = headers

metric.to_csv(file_path +  "/6 Traditional Thresh Results/"+ 'Metric_traditional.csv')


print(metric)

Recomputing the metric scores